In [ ]:
import torch
import sys
import glob
from tqdm.auto import tqdm, trange
sys.path.append("..")
sys.path.append("../..")
import matplotlib.pyplot as plt
from hist_optimizer import tensor_to_param_container, mse_engines_comparison, find_good_offset_problem, optimize_smart_walker, optimize_brute, optimize_pso, optimize_ea, evaluate_evaluation_method, plot_param_tensors, tensor_list_to_param_container_list, param_tensor_to_ray_outputs, compare_with_reference, fancy_plot_param_tensors

from ray_tools.base.transform import MultiLayer
from ray_tools.base.engine import RayEngine
from ray_nn.data.lightning_data_module import DefaultDataModule
from ray_nn.nn.xy_hist_data_models import MetrixXYHistSurrogate, StandardizeXYHist, HistSurrogateEngine, Model
from ray_tools.base.backend import RayBackendDockerRAYUI
from ray_tools.simulation.torch_datasets import BalancedMemoryDataset, MemoryDataset, RayDataset
from ray_optim.plot import Plot
from ray_tools.base.transform import Histogram, RayTransformConcat, XYHistogram
from ray_nn.data.transform import Select
from ray_tools.base.parameter import NumericalParameter, OutputParameter, NumericalOutputParameter, MutableParameter, RayParameterContainer
from sub_projects.ray_optimization.real_data import import_data
from sub_projects.ray_optimization.utils import ray_dict_to_tensor, ray_output_to_tensor

%load_ext autoreload
%autoreload 2
%matplotlib inline

# Model

In [ ]:
engine = RayEngine(rml_basefile='../../rml_src/METRIX_U41_G1_H1_318eV_PS_MLearn_1.15.rml',
                                exported_planes=["ImagePlane"],
                                ray_backend=RayBackendDockerRAYUI(docker_image='ray-ui-service',
                                                                  docker_container_name='ray-ui-service-test',
                                                                  dockerfile_path='../../ray_docker/rayui',
                                                                  ray_workdir='/dev/shm/ray-workdir',
                                                                  verbose=False),
                                num_workers=-1,
                                as_generator=False)


model_path = "../../outputs/xy_hist/ft1rr9h0/checkpoints/epoch=68-step=65665644.ckpt"
surrogate_engine = HistSurrogateEngine(checkpoint_path=model_path)

model = Model(path=model_path)

In [ ]:
%matplotlib widget
from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt

def f(X, Y, Z):
    inp = torch.ones((1,model.mutable_parameter_count), device=model.device)*0.5
    inp[:,-3] = X
    inp[:,-2] = Y
    inp[:,-4] = Z
    ##out = model(inp)
    out = surrogate_engine.run([tensor_to_param_container(inp[0], model.input_parameter_container)])
    #out =  engine.run([tensor_to_param_container(inp[0], model.input_parameter_container)])
    #out = [XYHistogram(50, (-10., 10.), (-3., 3.))(out[0]['ray_output']['ImagePlane'])['histogram'].flatten()]
    out = [torch.cat((out[0]['ray_output']['ImagePlane']['xy_hist'].x_loc, out[0]['ray_output']['ImagePlane']['xy_hist'].y_loc))]
    return out[0].detach().cpu().numpy()#A*x**2 + B*x + Z

#fig = plt.figure()
fig, ax = plt.subplots(1, 2)
histogram_lims = model.histogram_lims
lineX, = ax[0].plot(np.linspace(histogram_lims[0][0], histogram_lims[0][1], num=50), f(X=.5, Y=.5, Z=.0)[:50])
lineY, = ax[1].plot(np.linspace(histogram_lims[1][0], histogram_lims[1][1], num=50), f(X=.5, Y=.5, Z=.0)[50:])

def update(X = .5, Y = .5, Z = .0):
    new_f = f(X,Y,Z)
    new_x = new_f[:50]
    lineX.set_ydata(new_x)
    ax[0].set_ylim(new_x.min(), new_x.max())
    new_y = new_f[50:]
    lineY.set_ydata(new_y)
    ax[1].set_ylim(new_y.min(), new_y.max())
    fig.canvas.draw_idle()
    
interact(update, X = (0,1,0.05), Y = (0,1,0.05), Z = (0,1,0.1));

In [ ]:
inp_list = []
for i in range(20):
    inp = torch.ones(model.mutable_parameter_count, device=model.device)*0.5
    #inp[-3] = 0.5# + i*0.5
    inp[-4] = 0. + i*0.05
    inp_list.append(inp)

#Plot.plot_engines_comparison(engine, surrogate_engine, [tensor_to_param_container(inp, model.input_parameter_container) for inp in inp_list], MultiLayer([0.]))
inp = torch.stack(inp_list)
plot_param_tensors( inp, inp, engine = engine, ray_parameter_container=model.input_parameter_container, compensated_parameters = inp)

In [ ]:
out = model(inp)
plotted, _ = out.max(dim=-1)
plt.plot(plotted.cpu())

# Look with model for new sample

In [ ]:
offsets_selected, uncompensated_parameters_selected, compensated_parameters_selected = find_good_offset_problem(model, fixed_parameters = [8, 14, 20, 21, 27, 28])

with torch.no_grad():
    observed_rays = model(compensated_parameters_selected)

In [ ]:
offsets_list = []
uncompensated_parameters_list = []
compensated_parameters_list = []
for i in range(50):
    offsets_selected, uncompensated_parameters_selected, compensated_parameters_selected = find_good_offset_problem(model)
    offsets_list.append(offsets_selected)
    uncompensated_parameters_list.append(uncompensated_parameters_selected)
    compensated_parameters_list.append(compensated_parameters_selected)

offsets_list = torch.stack(offsets_list)
print(offsets_selected.mean().item(), "±", offsets_selected.std().item())

In [ ]:
import matplotlib.pyplot as plt
data = uncompensated_parameters_selected[:, 0, 0, :]
fig, ax = plt.subplots(2,1, figsize=(32,8))

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Plot each line with a different color from the cycle
for i, y in enumerate(uncompensated_parameters_list):
    ax[0].plot(y[:, 10, 0].T.cpu(), color=colors[i % len(colors)], alpha=0.2)
    ax[0].set_title("Uncompensated parameters")
    ax[0].set_xlabel("Beamline parameter [#]")
    ax[0].set_xticks(torch.arange(37))
    ax[1].plot(compensated_parameters_list[i][:, 10, 0].T.cpu(), color=colors[i % len(colors)], alpha=0.2)
    ax[1].set_title("Compensated parameters")
    ax[1].set_xlabel("Beamline parameter [#]")
    ax[1].set_xticks(torch.arange(37))
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1,2, sharey=True)
for entry in offsets_list:
    ax[0].scatter(entry[0].detach().cpu(), y=torch.arange(37).detach().cpu())
    ax[0].set_ylabel("Beamline parameter [#]")
    ax[0].set_xlabel("Chosen offset")
    ax[0].set_title("Simulated real beamline")
for entry in offsets_list:
    ax[1].scatter(torch.rand(37).detach().cpu(), y=torch.arange(37).detach().cpu())
    ax[1].set_xlabel("Chosen offset")
    ax[1].set_title("Random")

In [ ]:
from scipy.stats import kstest

def check_if_uniform(check_list):
    for i, entry in enumerate(check_list.T):
    # Example data
        data = entry.cpu() # entrynp.random.randn((10)) #[0.1, 0.2, 0.35, 0.9, 0.65]  # Replace with your data
        
        # Perform K-S test against uniform distribution
        statistic, p_value = kstest(data, 'uniform', args=(0, 1))
        
        #print(f"K-S statistic: {statistic}")
        #print(f"P-value: {p_value}")
        
        # Interpret p-value
        if p_value > 0.05:  # Common significance level is 0.05
            #print("Fail to reject the null hypothesis: Data appears to be uniformly distributed.")
            continue
        else:
            print(i, "Reject the null hypothesis: Data does not appear to be uniformly distributed.")

check_if_uniform(offsets_list.squeeze())

In [ ]:
mins, _ = uncompensated_parameters_selected[:, 0, 0, :].max(dim=0)
print((uncompensated_parameters_selected[:, 0, 0, :] == mins).sum(dim=0))

In [ ]:
for label, entry in model.offset_space.items():
    if isinstance(entry, MutableParameter):
        value_list = [value.value_lims for label2, value in model.input_parameter_container.items() if isinstance(value, MutableParameter) and label==label2]
        print("label:", entry.value_lims, "from", value_list[0])

In [ ]:
loss_min_params, loss, loss_min_list = optimize_smart_walker(model, observed_rays, uncompensated_parameters_selected, iterations=1000, num_candidates=10000, step_width=0.02)

In [ ]:
import torch
import numpy as np
from scipy.optimize import basinhopping

def torch_to_numpy(tensor):
    return tensor.detach().cpu().numpy()

def numpy_to_torch(array, device):
    return torch.tensor(array, device=device, dtype=torch.float32)

def objective_function(offsets_numpy, model, observed_rays, uncompensated_parameters):
    # Convert numpy array to torch tensor
    offsets = numpy_to_torch(offsets_numpy, device=model.device).unsqueeze(0)
    #offsets = torch.clamp(offsets, 0, 1)
    
    # Evaluate the model's loss with these offsets
    scaled_offsets = model.rescale_offset(offsets)
    compensated_rays = model(uncompensated_parameters + scaled_offsets)
    loss = ((compensated_rays - observed_rays) ** 2).mean().item()
    return loss

def optimize_smart_walker_scipy(model, observed_rays, uncompensated_parameters, iterations=100, hop_step=0.1, interval=1, stepsize=0.1):
    # Convert initial parameters to numpy for scipy
    initial_offsets = torch.rand(1, uncompensated_parameters.shape[-1], device=model.device)
    initial_offsets_numpy = torch_to_numpy(initial_offsets).flatten()

    # Define a list to track the minimum loss for each iteration
    loss_min_list = []

    # Set up the progress bar
    pbar = tqdm(total=iterations)

    def callback(x, f, accept):
        # Update progress bar and log the loss
        loss_min_list.append(f)
        pbar.update(1)
        pbar.set_postfix({"loss": f})

    # Configure basinhopping with scipy
    minimizer_kwargs = {
        "method": "Powell",
        "args": (model, observed_rays, uncompensated_parameters),
        "bounds": [(0, 1)] * initial_offsets_numpy.size  # Bounds for each parameter
    }
    
    # Run basinhopping
    result = basinhopping(
        objective_function, 
        initial_offsets_numpy, 
        niter=iterations,
        T=hop_step,
        stepsize=stepsize,
        interval=interval,
        minimizer_kwargs=minimizer_kwargs,
        callback=callback,
        disp=True
    )

    # Close the progress bar
    pbar.close()
    
    # Convert the final result back to torch for further use
    final_offsets_numpy = result.x
    final_offsets_torch = numpy_to_torch(final_offsets_numpy, device=model.device).view(1, -1)
    scaled_offsets = model.rescale_offset(final_offsets_torch.unsqueeze(1))
    best_loss_params = uncompensated_parameters + scaled_offsets

    return best_loss_params, result.fun, loss_min_list
loss_min_params, loss, loss_min_list = optimize_smart_walker_scipy(model, observed_rays, uncompensated_parameters_selected, hop_step=0.1, iterations=1000, interval=10, stepsize=.01)

In [ ]:
import optuna

optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective_function(offsets, model, observed_rays, uncompensated_parameters):
    # Evaluate the model's loss with these offsets
    scaled_offsets = model.rescale_offset(offsets)
    compensated_rays = model(uncompensated_parameters + scaled_offsets)
    loss = ((compensated_rays - observed_rays) ** 2).mean().item()
    return loss

def objective(trial):
    param_list = torch.tensor([trial.suggest_float(str(i), 0, 1.) for i in range(37)], device=model.device)
    return objective_function(param_list, model, observed_rays, uncompensated_parameters_selected)

study = optuna.create_study()

study.optimize(objective, n_trials=1000, show_progress_bar=True)

study.best_params  # E.g. {'x': 2.002108042}

In [ ]:
from ax import optimize
import torch
from ax.modelbridge.dispatch_utils import choose_generation_strategy
from ax.modelbridge.generation_strategy import GenerationStep, GenerationStrategy
from ax.modelbridge.modelbridge_utils import get_pending_observation_features
from ax.modelbridge.registry import ModelRegistryBase, Models

from ax.utils.testing.core_stubs import get_branin_experiment, get_branin_search_space

def optimize_smart_walker_ax(model, observed_rays, uncompensated_parameters, iterations, num_initial_points=10):
    """
    Optimize the smart walker using Bayesian Optimization via the Ax library.
    """
    device = model.device

    # Define the objective function
    def objective_function(parameterization):
        # Convert parameterization to tensor
        offsets = torch.tensor([parameterization[f"x{i}"] for i in range(uncompensated_parameters.shape[-1])], device=device).unsqueeze(0)
        
        # Scale offsets and compute the compensated rays
        scaled_offsets = model.rescale_offset(offsets)
        compensated_rays = model(uncompensated_parameters + scaled_offsets)
        
        # Calculate the loss (mean squared error)
        loss = ((compensated_rays - observed_rays) ** 2).mean().item()
        print(loss)
        return loss #{"loss": loss}  # Return loss as the objective value

    # Define the parameter space for optimization
    parameter_space = [
        {"name": f"x{i}", "type": "range", "bounds": [0.0, 1.0], "value_type": "float"} 
        for i in range(uncompensated_parameters.shape[-1])
    ]
    gs = GenerationStrategy(
        steps=[
            # 1. Initialization step (does not require pre-existing data and is well-suited for
            # initial sampling of the search space)
            GenerationStep(
                model=Models.SOBOL,
                num_trials=5,  # How many trials should be produced from this generation step
                min_trials_observed=3,  # How many trials need to be completed to move to next model
                max_parallelism=5,  # Max parallelism for this step
                model_kwargs={"seed": 999},  # Any kwargs you want passed into the model
                model_gen_kwargs={},  # Any kwargs you want passed to `modelbridge.gen`
            ),
            # 2. Bayesian optimization step (requires data obtained from previous phase and learns
            # from all data available at the time of each new candidate generation call)
            GenerationStep(
                model=Models.BOTORCH_MODULAR,
                num_trials=-1,  # No limitation on how many trials should be produced from this step
                max_parallelism=3,  # Parallelism limit for this step, often lower than for Sobol
                # More on parallelism vs. required samples in BayesOpt:
                # https://ax.dev/docs/bayesopt.html#tradeoff-between-parallelism-and-total-number-of-trials
            ),
        ]
    )
    gs = choose_generation_strategy(
    # Required arguments:
    search_space=get_branin_search_space(),  # Ax `SearchSpace`
    # Some optional arguments (shown with their defaults), see API docs for more settings:
    # https://ax.dev/api/modelbridge.html#module-ax.modelbridge.dispatch_utils
    use_batch_trials=False,  # Whether this GS will be used to generate 1-arm `Trial`-s or `BatchTrials`
    no_bayesian_optimization=False,  # Use quasi-random candidate generation without BayesOpt
    max_parallelism_override=None,  # Integer, to which to set the `max_parallelism` setting of all steps in this GS
    )

    # Use Ax to optimize the objective function
    best_parameters, values, experiment, surrogate = optimize(
        parameters=parameter_space,
        evaluation_function=objective_function,
        minimize=True,  # Minimize the loss
        total_trials=iterations,
        random_seed=42,  # Ensure reproducibility
        #init_trials=num_initial_points,  # Number of random initial samples
        generation_strategy = gs
    )

    # Extract the best found parameters and convert to tensor
    best_offsets = torch.tensor([best_parameters[f"x{i}"] for i in range(uncompensated_parameters.shape[-1])], device=device).unsqueeze(0)
    scaled_offsets = model.rescale_offset(best_offsets)
    loss_min_params = uncompensated_parameters + scaled_offsets
    loss_min = values[0]

    return loss_min_params, loss_min, experiment

loss_min_params, loss, loss_min_list = optimize_smart_walker_ax(model, observed_rays, uncompensated_parameters_selected, iterations=1000, num_initial_points=10)

In [ ]:
loss_min_params, loss, loss_min_list = optimize_pso(model, observed_rays, uncompensated_parameters_selected, iterations=1000, num_candidates=1000)

In [ ]:
loss_min_params, loss, loss_min_list = optimize_ea(model, observed_rays, uncompensated_parameters_selected, iterations=1000, num_candidates=1000)

In [ ]:
loss_min_params, loss, loss_min_list = optimize_brute(model, observed_rays, uncompensated_parameters_selected, iterations=1000, num_candidates=10000)

In [ ]:
loss_min_params, loss, loss_min_list = optimize_smart_walker(model, observed_rays, uncompensated_parameters_selected, iterations = 1000, num_candidates=10000)

In [ ]:
#fig = plot_param_tensors(loss_min_params[[1,2,4,8]], uncompensated_parameters_selected[[1,2,4,8]], engine = engine, ray_parameter_container=model.input_parameter_container, compensated_parameters=compensated_parameters_selected[[1,2,4,8]])

method_dict = {"smart walker": (optimize_smart_walker, 1000), "brute": (optimize_brute, 1000), "pso": (optimize_pso, 1000), "ea": (optimize_ea, 1000)}
method_evaluation_list = []

for key, entry in tqdm(method_dict.items(), desc="Evaluating methods"):
    mean_best, std_best, mean_progress, std_progress, loss_min_params_tens = evaluate_evaluation_method(entry[0], model, observed_rays, uncompensated_parameters_selected, offsets_selected, repetitions=2, num_candidates=entry[1], iterations=1000)
    method_evaluation_list.append((key, mean_best, std_best, mean_progress, std_progress))

    # calculate deviations from target offset
    #print("offsets", offsets_selected.shape)
    #print("loss_min_params_tens", loss_min_params_tens.shape, "uncompensated_parameters_selected", uncompensated_parameters_selected[0].shape)
    predicted_offsets = (loss_min_params_tens[:, 0] - uncompensated_parameters_selected[0, 0])
    #print(predicted_offsets.min(), predicted_offsets.max())
    normalized_predicted_offsets = model.unscale_offset(predicted_offsets)
    #print(normalized_predicted_offsets.min(), normalized_predicted_offsets.max())
    rmse = ((offsets_selected-normalized_predicted_offsets)**2).mean().sqrt().item()
    #print(key, ":", mean_best, "±", std_best, "RMSE from target offset:", rmse)
    #print("unc-pred",uncompensated_parameters_selected.shape, predicted_offsets.shape)
    loss_min_params_tens = (uncompensated_parameters_selected + predicted_offsets).swapaxes(0,1)
    #print(loss_min_params_tens.flatten(start_dim=0, end_dim=1))
    loss_min_params_tens = loss_min_params_tens.flatten(start_dim=0, end_dim=1)
    continue
    loss_min_ray_outputs = param_tensor_to_ray_outputs(loss_min_params_tens, engine, model.input_parameter_container)
    reference_ray_outputs = param_tensor_to_ray_outputs(compensated_parameters_selected.swapaxes(0,1), engine, model.input_parameter_container)
    compensated_parameters_selected_ray_outputs = param_tensor_to_ray_outputs(compensated_parameters_selected.swapaxes(0,1).repeat_interleave(10, dim=0), engine, model.input_parameter_container)
    out = compare_with_reference(reference_ray_outputs, loss_min_ray_outputs)
    print("deviation best to ref", out[0], "±", out[1])
    out = compare_with_reference(reference_ray_outputs, compensated_parameters_selected_ray_outputs)
    print("deviation ref to ref", out[0], "±", out[1])

In [ ]:
import torch
import matplotlib.pyplot as plt
from geomloss import SamplesLoss  # GeomLoss for Sinkhorn distance

# Step 1: Create two Gaussian point clouds
def create_gaussian_blob(center, cov, num_points=100):
    mean = center
    cov_matrix = torch.diag(torch.tensor(cov))  # Covariance matrix for Gaussian
    distribution = torch.distributions.MultivariateNormal(mean, cov_matrix)
    points = distribution.sample((num_points,))
    return points

# Create two Gaussian blobs
blob1 = create_gaussian_blob(center=torch.tensor([0.0, 0.0]), cov=[1.0, 1.0], num_points=100)
blob2 = create_gaussian_blob(center=torch.tensor([3.0, 3.0]), cov=[1.0, 1.0], num_points=100)

# Step 2: Define the Sinkhorn distance using GeomLoss
loss_fn = SamplesLoss("sinkhorn", blur=0.1)  # Sinkhorn distance with regularization

# Step 3: Compute the Sinkhorn distance
sinkhorn_distance = loss_fn(blob1, blob2)

# Print the resulting Sinkhorn distance
print(f"Sinkhorn distance between the two blobs: {sinkhorn_distance.item()}")

# Step 4: (Optional) Visualize the point clouds
plt.scatter(blob1[:, 0].cpu(), blob1[:, 1].cpu(), color='red', label='Blob 1', alpha=0.6)
plt.scatter(blob2[:, 0].cpu(), blob2[:, 1].cpu(), color='blue', label='Blob 2', alpha=0.6)
plt.title("2D Gaussian Blobs")

plt.legend()
plt.show()

In [ ]:
sl = SinkhornLoss()
sl.loss_fn(out[0], out[1], 'ImagePlane')

In [ ]:
plt.figure(figsize = (6.905, 4.434))
ax = plt.gca()
i = 0
plot_list = []
for key, mean_best, std_best, mean_progress, std_progress in method_evaluation_list:
    color = plt.rcParams["axes.prop_cycle"].by_key()["color"][i+3]
    plt.fill_between(torch.arange(len(mean_progress)), (mean_progress-std_progress).cpu(), (mean_progress+std_progress).cpu(), color=color, alpha=0.2)
    plot, = plt.plot(torch.arange(len(mean_progress)), mean_progress.cpu(), alpha = 1., c = color)
    plot_list.append(plot)
    i = i+1
ax.legend(plot_list, [key for key in method_dict.keys()], prop={'size': 11})
ax.tick_params(axis='both', which='major', labelsize=11)
plt.xlabel('Iteration', fontsize=16)
plt.ylabel('MSE (log)', fontsize=16)
ax.set_yscale('log')
plt.tight_layout()
plt.savefig('../../outputs/bl_optimizer_iterations.pdf', bbox_inches='tight', pad_inches = 0)
plt.show()

In [ ]:
t0 = benchmark.Timer(
    stmt='optimize_smart_walker(model, observed_rays, uncompensated_parameters_selected)',
    setup='from __main__ import optimize_smart_walker',
    globals={'model': model, 'observed_rays': observed_rays, 'uncompensated_parameters_selected': uncompensated_parameters_selected},
    num_threads=1,
    label='optimize smart walker',
    sub_label='optimize smart walker')
print(t0.timeit(repetitions))

In [ ]:
plot_observed_rays = observed_rays.squeeze()
plot_min_param_rays = model(loss_min_params)
fig, ax = plt.subplots(1, plot_observed_rays.shape[0], sharex=True, sharey=True, figsize=(32, 9))
with torch.no_grad():
    for i in range(plot_observed_rays.shape[0]):
        ax[i].plot(model(loss_min_params)[i].cpu())
        ax[i].plot(observed_rays.squeeze()[i].cpu())

In [ ]:
loss_min_params.shape
from hist_optimizer import simulate_param_tensor

out = [simulate_param_tensor(loss_min_params[:, i], engine, model.input_parameter_container) for i in range(loss_min_params.shape[1])]

In [ ]:
plot_param_tensors(loss_min_params[:,0], uncompensated_parameters_selected[:,0], engine = engine, ray_parameter_container=model.input_parameter_container, compensated_parameters=compensated_parameters_selected[:,0], )

In [ ]:
# input_param_tensor: tensor with dim [beamline_configs, z_layers, beamline_parameters]
from sub_projects.ray_optimization.utils import ray_output_to_tensor, ray_dict_to_tensor
def simulate_param_tensor(input_param_tensor, engine, ray_parameter_container, exported_plane='ImagePlane'):
    assert len(input_param_tensor.shape) == 3
    pc = tensor_list_to_param_container_list(input_param_tensor[:,0], ray_parameter_container)
    for entry in pc:
        entry[exported_plane+'.translationZerror'].value = 0.

    z_min, z_max = ray_parameter_container[exported_plane+'.translationZerror'].value_lims
    z_layers = input_param_tensor[0,:,-1] * (z_max-z_min) + z_min
    z_layer_list = z_layers.tolist()
    
    engine_output = engine.run(pc, MultiLayer(z_layer_list))
    return [ray_dict_to_tensor(entry, "ImagePlane", to_cpu=True) for entry in engine_output]
out = simulate_param_tensor(loss_min_params, engine, ray_parameter_container=model.input_parameter_container)

In [ ]:
fig = fancy_plot_param_tensors(loss_min_params[:], uncompensated_parameters_selected[:].squeeze(), engine = engine, ray_parameter_container=model.input_parameter_container, compensated_parameters=compensated_parameters_selected[:].squeeze())
import plotly.io as pio

# Save the figure to an HTML file
pio.write_html(fig, 'figure.html')

In [ ]:
from ray_tools.base.transform import Histogram, RayTransformConcat, XYHistogram

transforms = [
        XYHistogram(50, (-10., 10.), (-3., 3.))
]

fig, ax = plt.subplots(1,2, sharey=True, squeeze=False)

x_simulation_hist_list = []
y_simulation_hist_list = []
for i in trange(2):
    out = engine.run(tensor_list_to_param_container_list(loss_min_params), XYHistogram(50, (-10., 10.), (-3., 3.)))
    out_simulation = out[-1]['ray_output']['ImagePlane']['0.0']
    x_simulation_hist, _ = torch.histogram(out_simulation.x_loc,bins=50, range=[-10, 10])
    y_simulation_hist, _ = torch.histogram(out_simulation.y_loc,bins=50, range=[-3, 3])
    x_simulation_hist_list.append(x_simulation_hist / 22594.)
    y_simulation_hist_list.append(y_simulation_hist / 22594.)
    
    ax[0, 0].plot(torch.linspace(-10, 10, 50), x_simulation_hist / 22594.)
    ax[0, 1].plot(torch.linspace(-3, 3, 50), y_simulation_hist / 22594.)

In [ ]:
loss_min_params.shape
t = tensor_list_to_param_container_list(loss_min_params)
out = [engine.run(tensor_list_to_param_container_list(loss_min_params), XYHistogram(50, (-10., 10.), (-3., 3.))) for i in trange(20)]

In [ ]:
out[0][0]

In [ ]:
hist_list = []
for i in out:
    hist_list.append(torch.vstack([j['ray_output']['ImagePlane']['histogram'].reshape(-1) / 22594. for j in i]))
#        plt.plot(j['ray_output']['ImagePlane']['histogram'].reshape(-1) / 22594.)

In [ ]:
hist_list_tensor = torch.stack(hist_list)
#print(hist_list_tensor.shape)
print("var",hist_list_tensor.var(dim=0, correction=0).mean().item())

# Datamodule

In [ ]:
load_len: int | None = None #10000
h5_files = list(glob.iglob('datasets/metrix_simulation/ray_emergency_surrogate_50+50+z/histogram_*.h5'))
    sub_groups = ['parameters', 'histogram/ImagePlane', 'n_rays/ImagePlane']
    transforms=[lambda x: x[1:].float(), lambda x: standardizer(x.flatten().float()), lambda x: x.int()]
    dataset = HistDataset(h5_files, sub_groups, transforms, normalize_sub_groups=['parameters'], load_max=load_len)


bal_memory_dataset = BalancedMemoryDataset(dataset=dataset, load_len=load_len, min_n_rays=1)
memory_dataset = MemoryDataset(dataset=dataset, load_len=load_len)
datamodule = DefaultDataModule(dataset=bal_memory_dataset, num_workers=4)
datamodule.prepare_data()
datamodule.setup(stage="test")
test_dl = datamodule.test_dataloader()

unbal_datamodule = DefaultDataModule(dataset=memory_dataset, num_workers=4)
unbal_datamodule.prepare_data()
unbal_datamodule.setup(stage="test")
unbal_test_dl = unbal_datamodule.test_dataloader()

## Maximum distribution

In [ ]:
value_list = []
params_list = []

for i in tqdm(unbal_test_dl):
    biggest = i[1].flatten(start_dim=1)
    biggest, _ = i[1].flatten(start_dim=1).max(dim=1)
    mask = biggest > 0.8
    value_list.append(biggest[mask])
    params_list.append(i[0][mask])
value_tensor = torch.cat(value_list)
params_tensor = torch.cat(params_list)

torch.save(value_tensor, 'outputs/values.pt')
torch.save(params_tensor, 'outputs/params.pt')
plt.hist(value_tensor)
plt.savefig('outputs/max_dist_hist.png')

In [ ]:
value_tensor = torch.load('outputs/values.pt')
params_tensor = torch.load('outputs/params.pt')

In [ ]:
for i in tqdm(test_dl):
    #print(len(i[0][:10]))
    t = tensor_list_to_param_container_list(i[0][:10])
    print(len(t))
    out = [engine.run(t, XYHistogram(50, (-10., 10.), (-3., 3.))) for i in trange(2)]
    break
#['ray_output']['ImagePlane']['histogram']


In [ ]:
print(out[0][0]['ray_output']['ImagePlane']['histogram'])
len(out), len(out[0])

# Special sample

In [ ]:
with open("outputs/special_sample_168_selected.pkl", "rb") as f:
    special_sample = pickle.load(f, fix_imports=True, encoding='ASCII', errors='strict', buffers=None)
observed_params = special_sample.uncompensated_parameters

for param_container in observed_params:
    for label in ['ImagePlane.translationXerror', 'ImagePlane.translationYerror', 'ImagePlane.translationZerror']:
        if label in list(param_container.keys()):
            del param_container[label]

In [ ]:
len(observed_params)
Plot.plot_engines_comparison(engine, surrogate_engine, observed_params[:5], MultiLayer([0.]), )

In [ ]:
uncompensated_parameters = [elem.clone() for elem in special_sample.uncompensated_parameters]
for elem in uncompensated_parameters:
    elem.perturb(special_sample.target_params)
uncompensated_parameters[0]

In [ ]:
Plot.plot_engines_comparison(engine, surrogate_engine, uncompensated_parameters, MultiLayer([0.]))

In [ ]:
mse_comparison, x_simulation_hist, y_simulation_hist = mse_engines_comparison(engine, surrogate_engine, uncompensated_parameters[:5], MultiLayer([0.]))
plt.clf()
fig, ax = plt.subplots(1, 3)
for hist in x_simulation_hist:
    ax[0].plot(hist, alpha=0.3)
for hist in y_simulation_hist:
    ax[1].plot(hist, alpha=0.3)
ax[2].hist(mse_comparison)

In [ ]:
x_loc_list = []
good_param_list = []
batch_size = 5000
for i in trange(15000//batch_size):
    param_container = [tensor_to_param_container(torch.rand((34,))) for _ in range(batch_size)]
    surrogate_out = surrogate_engine.run(param_container, MultiLayer([0.]))
    for j in range(len(param_container)):
        output = surrogate_out[j]['ray_output']['ImagePlane']['xy_hist']
        if output.x_loc.sum() > 0.5:
            x_loc_list.append(output.x_loc.sum())
            good_param_list.append(param_container[j])

observed_containers_tensor = torch.vstack([surrogate_engine.select({"1e5/params":param_container})[0] for param_container in observed_params])
good_containers_tensor = torch.vstack([surrogate_engine.select({"1e5/params":param_container})[0] for param_container in good_param_list])

In [ ]:
plt.clf()
for i in range(good_containers_tensor.shape[0]):
    plt.plot(good_containers_tensor[i], c = 'blue', alpha=0.1)
for i in range(observed_containers_tensor.shape[0]):
    plt.plot(observed_containers_tensor[i], alpha=0.8)
plt.legend(["special", "good"])

In [ ]:
#mask = value_tensor > 0.44
out = ((params_tensor - observed_containers_tensor[0].unsqueeze(0))**2)/2.
out = out.mean(dim=1)
out_sorted, indices = torch.sort(out)
#part_indices = indices[:5]
#print(part_indices.shape)
#min_arg = out.argmin()
#plt.hist(out.mean(dim=1))
#plt.plot(params_tensor[min_arg])
#plt.plot(observed_containers_tensor[0])
for i in indices[:1]:
    plt.plot(params_tensor[i])
Plot.plot_engines_comparison(engine, surrogate_engine, [tensor_to_param_container(params_tensor[min_arg]) for min_arg in indices[:1]], MultiLayer([0.]), )

In [ ]:
transforms = {"ImagePlane": transform for transform in cfg_transforms}
out_engine = engine.run(observed_params, transforms)

In [ ]:
standardized_simulations = surrogate_engine.model.standardizer(torch.vstack([element['ray_output']['ImagePlane']['histogram'].flatten(start_dim=0) for element in out_engine]))
a = ((standardized_simulations - out_model)**2).mean(dim=1)

In [ ]:
plt.hist(a)

# Good params vs. bad params

In [ ]:
params_list = []
num_rays_list = []
for i in tqdm(memory_dataset):
    params_list.append(i[0])
    num_rays_list.append(i[2])
params_tensor = torch.vstack(params_list)
num_rays_tensor= torch.vstack(num_rays_list)
plt.hist(torch.tensor(num_rays_list))

In [ ]:
biggest = torch.tensor(num_rays_list).argmax()
test_parameters = memory_dataset[biggest][0]
param_container_list = [tensor_to_param_container(test_parameters)]

fig, ax = plt.subplots(1,2, sharey=True, squeeze=False)

x_simulation_hist_list = []
y_simulation_hist_list = []
for i in trange(20):
    out = engine.run(param_container_list, MultiLayer([0.]))
    out_simulation = out[-1]['ray_output']['ImagePlane']['0.0']
    x_simulation_hist, _ = torch.histogram(out_simulation.x_loc,bins=50, range=[-10, 10])
    y_simulation_hist, _ = torch.histogram(out_simulation.y_loc,bins=50, range=[-3, 3])
    x_simulation_hist_list.append(x_simulation_hist / 22594.)
    y_simulation_hist_list.append(y_simulation_hist / 22594.)
    
    ax[0, 0].plot(torch.linspace(-10, 10, 50), x_simulation_hist / 22594.)
    ax[0, 1].plot(torch.linspace(-3, 3, 50), y_simulation_hist / 22594.)


In [ ]:
x_simulation_hist_tens = torch.vstack(x_simulation_hist_list)
y_simulation_hist_tens = torch.vstack(y_simulation_hist_list)

In [ ]:
x_simulation_hist_tens.var(dim=0).mean()

In [ ]:
x_simulation_hist_tens.mean(dim=0).shape

In [ ]:
import umap
mask = num_rays_tensor > 100.
data_tensor = params_tensor[mask.flatten()]
data_tensor = data_tensor[:,:4]
class_tensor = num_rays_tensor[mask]

data_np = data_tensor.numpy()
class_np = class_tensor.numpy().flatten()

umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2)
umap_embedding = umap_model.fit_transform(data_np)

plt.figure(figsize=(12, 8))
scatter = plt.scatter(umap_embedding[:, 0], umap_embedding[:, 1], c=class_np, cmap='Spectral', s=5)
plt.colorbar(scatter)
plt.title('UMAP projection of the dataset')
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.show()

In [ ]:
from sklearn.manifold import TSNE
tsne_model = TSNE(n_components=2, perplexity=30, learning_rate=200, max_iter=1000)
tsne_embedding = tsne_model.fit_transform(data_np)

plt.figure(figsize=(12, 8))
scatter = plt.scatter(tsne_embedding[:, 0], tsne_embedding[:, 1], c=class_np, cmap='Spectral', s=5)
plt.colorbar(scatter)
plt.title('t-SNE projection of the dataset')
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.show()

In [ ]:
import glob
import tqdm
import matplotlib.pyplot as plt
import torch

from ray_nn.nn.xy_hist_data_models import MetrixXYHistSurrogate, StandardizeXYHist
from ray_tools.simulation.torch_datasets import MemoryDataset, HistDataset
from datasets.metrix_simulation.config_ray_emergency_surrogate import PARAM_CONTAINER_FUNC as params
from torch.utils.data import DataLoader
from ray_nn.data.transform import Select

model_path = "../../outputs/xy_hist/ee8cvj82/checkpoints/epoch=36-step=35212012.ckpt"
model = MetrixXYHistSurrogate.load_from_checkpoint(model_path)
model.to(torch.device('cpu'))
model.compile()
model.eval()

load_len: int | None = 1000
h5_files = list(glob.iglob('datasets/metrix_simulation/ray_emergency_surrogate_50+50+z/histogram_*.h5'))
sub_groups = ['parameters', 'histogram/ImagePlane', 'n_rays/ImagePlane']
transforms=[lambda x: x[1:].float(), lambda x: standardizer(x.flatten().float()), lambda x: x.int()]
dataset = HistDataset(h5_files, sub_groups, transforms, normalize_sub_groups=['parameters'], load_max=load_len)


memory_dataset = MemoryDataset(dataset=dataset, load_len=load_len)

train_dataloader = DataLoader(memory_dataset, batch_size=2048, shuffle=False, num_workers=0)

errors_list = []
with torch.no_grad():
    for par_input, label, _ in tqdm(train_dataloader):
        out = model(par_input)
        label = label.flatten(start_dim=1)
        b = ((label - out)**2).mean(dim=1)
        errors_list.append(b)
errors_tensor = torch.cat(errors_list)

plt.hist(errors_tensor)
plt.savefig('outputs/dataset_errors_hist.png')
torch.save(errors_tensor, 'outputs/dataset_errors.pt')

# Import real data

In [ ]:
def import_real_hist_data(parameter_container, device, path = '../../datasets/metrix_real_data/2021_march_complete', import_set = ['M03', 'M10', 'M18', 'M22', 'M23', 'M24', 'M25', 'M27', 'M28', 'M29', 'M30', 'M32', 'M33', 'M36',
                             'M37', 'M40', 'M41', 'M42', 'M43', 'M44'], z_layers=[0., 5., 10.], check_value_lims=False, z_array_label='ImagePlane.translationZerror'):
    imported_data = import_data(
                path,
                import_set,
                z_layers,
                parameter_container,
                check_value_lims=check_value_lims,
            )
    xy_hist = XYHistogram(50, (-10., 10.), (-3., 3.))
    z_array_min, z_array_max = model.input_parameter_container[z_array_label].value_lims
    normalized_z_array = torch.tensor((z_layers - z_array_min) / (z_array_max - z_array_min), device=device).float()

    # observed_rays_point_cloud
    real_data_point_cloud_list = []
    for i in range(len(imported_data)):
        real_data_point_cloud_list.append(imported_data[i])
    observed_rays_point_cloud = [ray_dict_to_tensor(entry, 'ImagePlane') for entry in real_data_point_cloud_list]

    real_data_list = []
    for i in range(len(imported_data)):
        real_data_list.append(xy_hist(imported_data[i]['ray_output']['ImagePlane']))
    z_layer_real_data_tensor_list = []
    for z in z_layers:
        z_layer_real_data_tensor_list.append(torch.stack([real_data_list[i][z]['histogram'] for i in range(len(real_data_list))]))
    real_data_tensor = torch.stack(z_layer_real_data_tensor_list, dim=1)

    real_data_tensor_normalized = real_data_tensor / 4000. #surrogate_engine.model.standardizer(real_data_tensor)
    observed_rays = real_data_tensor_normalized.flatten(start_dim=-2).unsqueeze(2).float().to(device)

    uncompensated_parameters_list = []
    for i in range(len(imported_data)):
        uncompensated_entry = torch.tensor([value.get_value() for value in Plot.normalize_parameters(imported_data[i]['param_container_dict'], parameter_container).values()])
        uncompensated_parameters_list.append(uncompensated_entry)
    uncompensated_parameters = torch.stack(uncompensated_parameters_list)
    uncompensated_parameters = uncompensated_parameters.unsqueeze(1).float().to(device)
    uncompensated_parameters = uncompensated_parameters.repeat_interleave(len(z_layers), dim=1).unsqueeze(2)
    uncompensated_parameters[:, :, 0, -1] = normalized_z_array
    
    #print(uncompensated_parameters)
    
    return observed_rays, uncompensated_parameters, observed_rays_point_cloud
    
observed_rays_real, uncompensated_parameters_real, observed_rays_point_cloud = import_real_hist_data(model.input_parameter_container, device=model.device)

In [ ]:
loss_min_params = optimize_smart_walker(model, observed_rays_real, uncompensated_parameters_real, iterations=1000, num_candidates=40000)

In [ ]:
torch.set_default_device('cpu')
pc = [tensor_to_param_container(loss_min_params[i].cpu()) for i in range(loss_min_params.shape[0])]
Plot.plot_engines_comparison(engine, surrogate_engine, pc[:8], MultiLayer([0.]), )

In [ ]:
plot_param_tensors(loss_min_params[:], uncompensated_parameters_selected[:], compensated_parameters=compensated_parameters_selected[:])

In [ ]:
repeated_params = torch.vstack(10*[loss_min_params[1]])
a = repeated_params.clone()
b = repeated_params.clone()
c = repeated_params.clone()
for i, ten in enumerate(a):
    ten[-1]=0.5+0.1*i
    ten[-2]=0.5
for i, ten in enumerate(b):
    ten[-2]=0.5+0.1*i
    ten[-1]=0.5
for i, ten in enumerate(c):
    ten[-1]=0.5
    ten[-2]=0.5#+0.005*i
plot_param_tensors(a[:3], b[:3], compensated_parameters=c[:3])
#print(repeated_params)
#plot_param_tensors(loss_min_params[:], uncompensated_parameters_selected[:], compensated_parameters=compensated_parameters_selected[:])

In [ ]:
loss_min_params.shape, uncompensated_parameters_selected.shape, compensated_parameters_selected.shape

In [ ]:
parameter_comparison_plot = Plot.plot_param_comparison(
    predicted_params=tensor_list_to_param_container_list(loss_min_params.squeeze().unsqueeze(0)),
    epoch=42,
    training_samples_count=len(observed_rays),
    search_space=model.offset_space,
    real_params=tensor_list_to_param_container_list(compensated_parameters_selected)[0],
)

In [ ]:
parameter_comparison_plot = Plot.plot_param_comparison(
    predicted_params=tensor_to_param_container(offsets_selected.squeeze()),
    epoch=42,
    training_samples_count=len(observed_rays),
    search_space=RayOptimization.limited_search_space(model.offset_space, RandomGenerator(42), max_deviation=max_offset),
    real_params=tensor_list_to_param_container_list(loss_min_params[0] - compensated_parameters_selected.squeeze())[1],
)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import transforms
import numpy as np
from matplotlib.layout_engine import ConstrainedLayoutEngine, TightLayoutEngine
from matplotlib.ticker import NullLocator
from matplotlib.ticker import FormatStrFormatter

rs = np.random.RandomState(11)
x = rs.gamma(4, size=1000)
y = -.5 * x + rs.normal(size=1000)
x = x / 4  -1
y = y/4 +1


def scatter_hist(x, y, ax, ax_histx, ax_histy):
    xlim_min, xlim_max, ylim_min, ylim_max = -2, 2, -2, 2
    # no labels
    ax_histx.tick_params(axis="x", labelbottom=False)
    ax_histy.tick_params(axis="y", labelleft=False)

    # the scatter plot:
    color = plt.rcParams["axes.prop_cycle"].by_key()["color"][1]
    ax.scatter(
    x,
    y,
    s=2.0,
    alpha=0.5,
    linewidths=0.4,
    color=color
    )
    ax.set_xlim(
    Plot.scale_interval(
        xlim_min, xlim_max, 1.2
    )
)
    ax.set_ylim(
        Plot.scale_interval(
            ylim_min, ylim_max, 1.2
        )
    )
    ax.tick_params(axis="both", length=0.0)
    ax.grid(linestyle="dashed", alpha=0.5)
    ax.xaxis.set_major_locator(NullLocator())
    ax.yaxis.set_major_locator(NullLocator())
    ax.xaxis.set_major_formatter(FormatStrFormatter("%.1f"))
    ax.yaxis.set_major_formatter(FormatStrFormatter("%.1f"))
    
    ax.set_xticks((xlim_min, xlim_max))
    ax.set_yticks((ylim_min, ylim_max))
    ax.set_yticklabels([])
    ax.set_xticklabels([])

    # now determine nice limits by hand:
    binwidth = 0.25
    xymax = max(np.max(np.abs(x)), np.max(np.abs(y)))
    lim = (int(xymax/binwidth) + 1) * binwidth

    bins = np.arange(-lim, lim + binwidth, binwidth)
    #ax_histx.hist(x, bins=bins, color=color)
    x_simulation_hist, _ = torch.histogram(torch.from_numpy(x),bins=50, range=[-2, 2])
    ax_histx.plot(torch.linspace(-2, 2, 50), x_simulation_hist, color=color)
    ax_histx.set_yticklabels([])
    #ax_histy.hist(y, bins=bins, orientation='horizontal', color=color)
    y_simulation_hist, _ = torch.histogram(torch.from_numpy(y),bins=50, range=[-2, 2])
    ax_histy.plot(y_simulation_hist, torch.linspace(-2, 2, 50), color=color)
    ax_histy.set_xticklabels([])


# Create a Figure, which doesn't have to be square.
fig = plt.figure(layout='constrained')
# Create the main Axes, leaving 25% of the figure space at the top and on the
# right to position marginals.
ax = fig.add_gridspec(top=0.75, right=0.75).subplots()
# The main Axes' aspect can be fixed.
ax.set(aspect=1)
# Create marginal Axes, which have 25% of the size of the main Axes.  Note that
# the inset Axes are positioned *outside* (on the right and the top) of the
# main Axes, by specifying axes coordinates greater than 1.  Axes coordinates
# less than 0 would likewise specify positions on the left and the bottom of
# the main Axes.
ax_histx = ax.inset_axes([0, 1.05, 1, 0.25], sharex=ax)
ax_histy = ax.inset_axes([1.05, 0, 0.25, 1], sharey=ax)
# Draw the scatter plot and marginals.
scatter_hist(x, y, ax, ax_histx, ax_histy)

plt.show()


# Evotorch

In [ ]:
from evotorch import Problem
from evotorch.algorithms import SNES
from evotorch.logging import StdOutLogger

def norm(x: torch.Tensor) -> torch.Tensor:
  return torch.linalg.norm(x, dim=-1)

with torch.no_grad():
    observed_rays = model(compensated_parameters_selected)
uncompensated_parameters = uncompensated_parameters_selected
def loss(x: torch.Tensor) -> torch.Tensor:
    with torch.no_grad():
        tensor_sum = model.rescale_offset(x) + uncompensated_parameters
        compensated_rays = model(tensor_sum, clone_output=True)
        #compensated_rays = compensated_rays.flatten(start_dim=2)
        loss_orig = ((compensated_rays - observed_rays) ** 2).mean(0).mean(0).mean(-1)
        #loss_orig = ((compensated_rays - observed_rays) ** 2).mean(0).mean(-1)
    return loss_orig

problem = Problem(
  "min",
  loss,
  initial_bounds=(0., 1.),
  solution_length=model.mutable_parameter_count,
  vectorized=True,
  device="cuda",  # Enable for GPU support
)

searcher = SNES(problem, popsize=1000, stdev_init=0.3)
_ = StdOutLogger(searcher, interval=1000)

searcher.run(num_generations=10000)

In [ ]:
best_discovered_solution = searcher.status["pop_best"]
print(best_discovered_solution.values)

# SGD

In [ ]:
batch_size = 10000
x = torch.rand((batch_size, uncompensated_parameters.shape[-1]), requires_grad=True, device=model.device)
optimize_algo = 'adam'
if optimize_algo == 'adam':
    optimizer = torch.optim.Adam([x], lr=0.01)  # You can adjust the learning rate as needed
else:
    optimizer = torch.optim.SGD([x], lr=0.01)  # Adjust the learning rate as needed
torch.autograd.set_detect_anomaly(True)
num_epochs = 1000  # Number of iterations
uncompensated_parameters = uncompensated_parameters_selected
#with torch.no_grad():
observed_rays = model(compensated_parameters_selected, clone_output=True, grad=True)

def function_to_minimize(x):
    tensor_sum = x + uncompensated_parameters
    compensated_rays = model(tensor_sum, clone_output=True, grad=True)
    compensated_rays = compensated_rays.flatten(start_dim=2)
    loss_orig = ((compensated_rays - observed_rays) ** 2).mean(0).mean(-1)
    return loss_orig

pbar = trange(num_epochs)
for epoch in pbar:
    optimizer.zero_grad()  # Clear the gradients from the previous step
    
    output = function_to_minimize(x.detach())  # Compute the function's output
    loss = output.mean()  # Compute the mean loss for this batch
    
    loss.backward(retain_graph=True)  # Backpropagate to compute the gradients
    optimizer.step()  # Update the parameters with SGD

    # Optionally print the loss to monitor progress
    if epoch % 100 == 0:
        pbar.set_postfix({"loss": loss.item()})

In [ ]:
def rescale(value, o1, o2, m1, m2):
    if o2 == o1:
        raise ValueError("The interval [x1, x2] cannot have zero width.")
    return m1 + (value - o1) * (m2 - m1) / (o2 - o1)

# Example usage:
o1, o2 = 0, 10  # original interval
m1, m2 = 100, 200  # new interval
value = 5  # value to rescale

rescaled_value = rescale(value, o1, o2, m1, m2)
print("Rescaled value:", rescaled_value)

In [ ]:
def rescale(value, o1, o2, m1, m2):
    rescale_scaler = 1. / (m2-m1)
    rescale_multiplier = (o2-o1) * rescale_scaler
    rescale_addend = (o1-m1) * rescale_scaler
    return value*rescale_multiplier+rescale_addend
    
# Example usage:
o1, o2 = -1, 1  # original interval
m1, m2 = -3, 3  # new interval
value = 0  # value to rescale

rescaled_value = rescale(value, o1, o2, m1, m2)
print("Rescaled value:", rescaled_value)


In [ ]:
a = [0., 1., 2., 3.]
torch.tensor(a)

In [ ]:
for key, value in model.offset_space.items():
    if isinstance(value, MutableParameter):
        print(key, value.value_lims)